<a href="https://colab.research.google.com/github/majorquev/DBDC_202106_Programacion_en_R_para_ciencia_de_datos/blob/main/Clases/Clase8/Clase8_3_Regresi%C3%B3n_log%C3%ADstica_en_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 9: Regresión logística en R

**Autor:** Miguel Jorquera Viguera.<br>
**E-mail:** majorquev@gmail.com<br>
**Ayudante:** Jorge Arévalo Labarca<br>
**E-mail ayudante:** jorge.arevalol@usach.cl

Si nuestro interés no es una variable cuantitativa, si no más bien categórica, una regresión lineal no es adecuado.
Por ejemplo, supongamos que nos interesa saber si una persona quedará en “default” (no terminará de pagar la deuda de su tarjeta de crédito), en función de atributos como su ingreso, nivel de deuda y si es estudiante o no.
Para tal efecto nos interesa modelar entonces la probabilidad de caer en “default”. Para modelar dicha probabilidad, es posible especificar una dependencia lineal entre una "función" de la probabilidad a estimar, en términos de una  combinación lineal de las variables explciativas. Tal función se denomina _función de enlace_, la que en el caso de la regresión logística tradicional, corresponde al _logaritmo de la chance_ del evento a estudiar. Esta especificación del modelo matemático se muestra a continuación.


## Especificación matemática del modelo

En el caso de la regresión logística, lo que se suele modelar linealmente es el logaritmo de la chance del vento de interés, donde la chance es simplemente la reazón entre la probabilidad de que ocurra el evento y la probabilidad de que no ocurra.

$$\log\left(\dfrac{P(Y=1 | \mathbf{X} = \mathbf{x})}{1- P(Y=1 | \mathbf{X} = \mathbf{x})}\right) = \beta_0 + \beta_1 X_1 + \dots + \beta_pX_p$$

Bajo esta especificación es posible estimar los parámetros mediante mínimos cuadrados ponderados.

## En R

En R la cosa es más fácil y no necesitamos tanta escritura matemática para ajustar un regresión logística, lo cual lo podemos llevar a cabo con la función `glm()`, cuya documentación podemos consultar.


Pongamos en práctica lo onversado en clases con el siguiente ejemplo.

### Dataset _default_

Para nuestro primer modelo logístico, cosideraremos el dataset _default_ el cual consta de una simulación de 200 registros que representa la información bancaria de 200 clientes. La información que contiene este dataset corresponde a  : 

 * `default`: Variable que indica si el cliente quedó ene stado de dafault ("Yes") o no ("No").
 * `income`: Ingreso promedio del cliente en una dererminada ventana de tiempo.
 * `student`: Variable que indica si el cliente es estudiante o no.
 * `balance`: Monto adeudado posterior al último pago de cuenta.
 
 
 

In [ ]:
library(dplyr)

In [ ]:
data <- read.csv("default.csv")
data <- data %>%
    mutate(Y = ifelse(default == "Yes",1,0))

In [ ]:
head(data)
tail(data)

In [ ]:
# Consideremos el modelo que incluye sólo el ingreso

mod1 <- glm(Y ~ income +balance + student,
          family = binomial(link = "logit"),
          data = data)

# Veamos un resumen del modelo

In [ ]:
summary(mod1)

In [ ]:
# Podemos generar predicciones con la función predict()
# Con el argumento type, controlamos el tipo de predicción a generar
    # response: para la probabilidad de Y = 1
    # terms: Combinación lineal de los predictores, con los coeficientes estimados. Se centran las variables con media 0.
    # link: Logaritmo de la chance (la evaluación del modelo linal).

new <- data.frame(income = c(44361.625,12106.135))
new




In [ ]:
?predict.glm

In [ ]:
predict(mod1, new, type = "response")
predict(mod1, type = "response")

In [ ]:
pred_train <- predict(mod1, type = "response")

In [ ]:
# Podemos analizar el desempeño de nuestro modelo fijando un punto de corte.
clase_pred <- ifelse(pred_train > 0.1, "Si", "No")  

In [ ]:
table(clase_pred, data$default) # matriz de confusión

## Odds ratios

Una de las ventajas de la regresión logística, es que es un modelo del cual se pueden extraer conclusiones interesantes a partir de sus coeficientes. Concretamente, la exponencial de los "beta" asociados a la regresión logística, llamados "odds ratios" cuantifican el cambio en la chance, cuando la variable asociada a dicho coeficiente cambia de estado (o aumenta en una unidad)

A modo de ejemplo, supongamos que estamos modelando la variable respuesta $Y$ (binaria) como función de la covariable $X$.

Recordemos que lo que se modela linealmente corresponde al logaritmo de la chance $chance(x):=\log\left(\dfrac{P(Y = 1 | X=x)}{1-P(Y = 1| X=x)}\right)$, donde $P(Y = 1 | X=x)$ denota la _probablidad de obervar el evento de interés dado el valor_ $X = x$ _de nuestra covariable_, entonces:


$$
chance(x):= \log\left(\dfrac{P(Y = 1 | X=x)}{1-P(Y = 1| X=x)}\right) =  \beta_0 + \beta_1x \ \ \ \ \ \ \ \ \ \ (1) 
$$

Consideremos ahora la chance de observar $Y = 1$ dado que $X=x+1$,
$$
chance(x+1) = \beta_0 + \beta_1(x+1) \ \ \ \ \ \ \ \ \ \ \ \ (2) 
$$

Si restamos (2) - (1), se obtiene
$$
\log\left(\dfrac{chance(x+1)}{chance(x)}\right) = \beta_1
$$


De donde podemos inferir que $\exp(\beta_1)$, corresponde al cambio en la chance de obervar $Y = 1$, cuando X aumenta en una unidad. La expresión $\exp(\beta_1)$, se conoce como razón de chances, o bien _odd ratio_ .

In [ ]:
# suponiendo que el modelo es un buen modelo, ¿Cómo podemos interpretar los coeficientes estimados?
# ¿Cómo varia la chance de caer en default cuado se aumenta en una unidad el balance.?

# la chance varía en un 
exp(5.737e-03)
#la chance aumenta en un 0.5%, lo que es equivalente a decir que la chance de caer en default cuando el balance aumenta en 1, es 1.0057 veces la chance original.